# Import Financial Performance Data via Alpha Vantage API

In [96]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from alpha_vantage.timeseries import TimeSeries 
import csv
import os

print("All libraries loaded")

All libraries loaded


Establish the API request parameters

In [186]:
config = {
    "alpha_vantage": {
        "key": "VFX0IYC0TKUCTIWI", # Claim your free API key here: https://www.alphavantage.co/support/#api-key
        "symbol": "ANFI",
        "outputsize": "full",
        "key_adjusted_close": "5. adjusted close",
    }
}

Define the API request

In [187]:
def download_data(config):
    ts = TimeSeries(key=config["alpha_vantage"]["key"], output_format='csv')
    data, meta_data = ts.get_daily_adjusted(config["alpha_vantage"]["symbol"], outputsize=config["alpha_vantage"]["outputsize"])

    return data

data = download_data(config)

Write returns to csv for each company

In [224]:
with open('Data/Returns/ANFI.csv', 'w') as write_csvfile:
    writer = csv.writer(write_csvfile, dialect='excel')
    for row in data:
        writer.writerow(row)

Manually import last decade of S&P 500 annual returns

In [189]:
# source: https://www.berkshirehathaway.com/letters/2019ltr.pdf
SP500 = [['SP500',2010, 0.151]
        ,['SP500',2011, 0.021]
        ,['SP500',2012, 0.16]
        ,['SP500',2013, 0.324]
        ,['SP500',2014, 0.137]
        ,['SP500',2015, 0.014]
        ,['SP500',2016, 0.12]
        ,['SP500',2017, 0.218]
        ,['SP500',2018, -0.044]
        ,['SP500',2019, 0.315]
        ,['SP500',2020, 0.184]]
 
# Create the pandas DataFrame
df_returns = pd.DataFrame(SP500, columns = ['Ticker','Year', 'Adj_Return'])

For each company, add annual returns to a master returns csv

In [191]:
tickers = ['ADM','ANFI','BGS','CAG','CPB','DAR','FLO','GIS'
            ,'INGR','K','LW','MKC','NOMD','POST','SJM','THS','TR','TSN','HSY']

stock_returns = []
stock_returns.append(df_returns)

for x in tickers:
    df = pd.read_csv('Data/Returns/{}.csv'.format(x),header=0)
    df.timestamp = pd.to_datetime(df.timestamp)
    df.set_index('timestamp',inplace=True)
    annual_returns = df['adjusted_close'].resample('Y').ffill().pct_change()
    annual_returns = annual_returns.to_frame()
    annual_returns.reset_index(level=0, inplace=True)
    annual_returns = annual_returns[annual_returns['timestamp'].dt.year >= 2010]
    annual_returns = annual_returns[annual_returns['timestamp'].dt.year <= 2020]
    annual_returns['timestamp'] = annual_returns['timestamp'].dt.year
    annual_returns['ticker'] = x
    annual_returns = annual_returns[['ticker','timestamp','adjusted_close']]
    annual_returns.rename(columns={'ticker':'Ticker','timestamp':'Year','adjusted_close':'Adj_Return'}, inplace=True)
    stock_returns.append(annual_returns)

return_data = pd.concat(stock_returns)
return_data.reset_index(level=0, inplace=True)
return_data.drop(columns=['index'], inplace=True)
return_data.to_csv('Data/all_returns.csv')

Compare returns to S&P

In [218]:
df_join = return_data[return_data['Ticker'] != 'SP500'].copy()
df_join.reset_index(level=0, inplace=True)
df_join.drop(columns=['index'], inplace=True)
df_join.head()

,Ticker,Year,Adj_Return
0,ADM,2010,-0.019293
1,ADM,2011,-0.029051
2,ADM,2012,-0.018320
3,ADM,2013,0.618549
4,ADM,2014,0.223251


Export returns data with 'Excess Returns' metric

In [223]:
input_data = df_join.merge(df_returns, left_on='Year', right_on='Year')
input_data['Excess_Return'] = input_data['Adj_Return_x'] - input_data['Adj_Return_y']
input_data.rename(columns={'Adj_Return_x':'Adj_Return','Adj_Return_y':'SP500_Return', 'Ticker_x':'Ticker'}, inplace=True)
input_data.drop(columns=['Ticker_y'], inplace=True)
input_data.to_csv('Data/returns_modeling.csv', index=False)